In [ ]:
import Oncoder
import torch
import pandas as pd
from Oncoder import Autoencoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Training Oncoder to learn a new methylation atlas based on simulation data and deconvoluting the simulation data.

In [ ]:
refdata = 'data/reference_data.tsv'
train_x,train_y = Oncoder.generate_simulated_data(refdata,prior=[0.8,0.2],samplenum=1000,random_state=1)
test_x,test_y = Oncoder.generate_simulated_data(refdata,prior=[0.8,0.2],samplenum=100,random_state=1)

In [ ]:
model=Oncoder.train_Oncoder(train_x,train_y,refdata,model_name='Oncoder_test',batch_size=128,epochs=256)

In [ ]:
pred_y,methyatlas = Oncoder.predict(test_x,model)

In [ ]:
pred_y

Loading Oncoder for predicting read data.

In [ ]:
model = torch.load('Oncoder.pth',map_location=device)

In [ ]:
Normal_data = pd.read_csv('data/normal_data.tsv',sep='\t',index_col=0,header=0).T.values
HCC_data = pd.read_csv('data/HCC_data.tsv',sep='\t',index_col=0,header=0).T.values

In [ ]:
pred_y,methyatlas = Oncoder.predict(Normal_data,model)

In [ ]:
pred_y